In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Load the model scores of feature importance #
model_score = pd.read_csv("RFoutputs/RFCompositeFeatureScore.csv", index_col=[0])
model_score

,Pair,Importance
0,47-74,0.046736
1,206-210,0.037477
2,200-206,0.036288
3,63-77,0.028305
4,208-271,0.025645
...,...,...
61,89-141,0.000121
62,121-303,0.000000
63,58-93,0.000000
64,288-294,0.000000


In [3]:
# Create an XYZ file for transforming into contact array for VMD NetworkView #
"""For two-body features like pairwise amino acid distance
   X = Res1; Y = Res2; Z = Feature Importance (could be any descriptive structural paramter)
"""
model_score[['Res1', 'Res2']] = model_score['Pair'].str.split('-', n=1, expand=True)
model_score[['Res1', 'Res2']] = model_score[['Res1', 'Res2']].astype(int).apply(lambda x: x - 27) # Renumber residue indices (PDB sequence) in VMD numbering format
model_score_xyz = model_score[['Res1', 'Res2', 'Importance']].copy()
model_score_xyz[['Importance']] = model_score_xyz[['Importance']].apply(lambda x: 1/(30*x))  # Scale feature importance suitably to compute edge weights compatible for network visualization
model_score_xyz

,Res1,Res2,Importance
0,20,47,0.713227
1,179,183,0.889444
2,173,179,0.918573
3,36,50,1.177638
4,181,244,1.299781
...,...,...,...
61,62,114,274.515642
62,94,276,inf
63,31,66,inf
64,261,267,inf


In [4]:
# Create an empty contact array of zeroes #
array = np.zeros([277, 277], dtype=float) # NxN matrix; N = No. of Nodes (atoms/residues)
array=pd.DataFrame(array)
array

,0,1,2,3,4,5,6,7,8,9,...,267,268,269,270,271,272,273,274,275,276
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Copy the DataFrame
array_new = array.copy()

# Loop through the residue indices 
for ind in model_score_xyz.index:
    val1 = int(model_score_xyz.loc[ind, 'Res2'])  # Ensure val1 is within bounds
    val2 = int(model_score_xyz.loc[ind, 'Res1'])  # Ensure val2 is within bounds
    val3 = model_score_xyz.loc[ind, 'Importance']

    # Check if indices are within bounds and append feature importance values into the matrix
    if 0 <= val1 < 277 and 0 <= val2 < 277:
        array_new.loc[val1, val2] = val3
    else:
        print(f"Warning: Out-of-bounds index ({val1}, {val2}) ignored")

print(array_new)
np.savetxt("RFoutputs/tmp.dat", array_new)

     0    1    2    3    4    5    6    7    8    9    ...  267  268  269  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
272  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
273  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
274  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
275  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
276  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

          270  271  272  273       274  275  276  
0    0.000000  0.0  0.0 

In [6]:
## Save the contact array (contact.dat) for loading in VMD NetworkView ##
open("RFoutputs/contact.dat", "w").write("((chain P) and (not hydrogen)) and ((name CA))\n" + open("RFoutputs/tmp.dat").read())
os.remove("RFoutputs/tmp.dat")